In [5]:
using Statistics
using Flux, Flux.Optimise
using Metalhead, Images
using Metalhead: trainimgs
using Images.ImageCore
using Flux: onehotbatch, onecold
using Base.Iterators: partition
#using CUDA
#https://fluxml.ai/tutorials/2020/09/15/deep-learning-flux.html

In [6]:
Metalhead.download(CIFAR10)
X = trainimgs(CIFAR10)
labels = onehotbatch([X[i].ground_truth.class for i in 1:50000],1:10)

x cifar-10-batches-bin/
x cifar-10-batches-bin/data_batch_1.bin
x cifar-10-batches-bin/batches.meta.txt
x cifar-10-batches-bin/data_batch_3.bin
x cifar-10-batches-bin/data_batch_4.bin
x cifar-10-batches-bin/test_batch.bin
x cifar-10-batches-bin/readme.html
x cifar-10-batches-bin/data_batch_5.bin
x cifar-10-batches-bin/data_batch_2.bin


10×50000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  1  1
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅     ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅

In [ ]:
image(x) = x.img # handy for use later
ground_truth(x) = x.ground_truth
image.(X[rand(1:end, 10)])

In [8]:
getarray(X) = float.(permutedims(channelview(X), (2, 3, 1)))
imgs = [getarray(X[i].img) for i in 1:50000]

50000-element Vector{Array{Float32, 3}}:
 [0.23137255 0.16862746 … 0.59607846 0.5803922; 0.0627451 0.0 … 0.46666667 0.47843137; … ; 0.7058824 0.6784314 … 0.38039216 0.3254902; 0.69411767 0.65882355 … 0.5921569 0.48235294]

[0.24313726 0.18039216 … 0.49019608 0.4862745; 0.078431375 0.0 … 0.3254902 0.34117648; … ; 0.54509807 0.48235294 … 0.24313726 0.20784314; 0.5647059 0.5058824 … 0.4627451 0.36078432]

[0.24705882 0.1764706 … 0.4 0.40392157; 0.078431375 0.0 … 0.19607843 0.22352941; … ; 0.3764706 0.16470589 … 0.13333334 0.13333334; 0.45490196 0.36862746 … 0.32941177 0.28235295]
 [0.6039216 0.49411765 … 0.34117648 0.30980393; 0.54901963 0.5686275 … 0.3019608 0.2784314; … ; 0.64705884 0.6117647 … 0.48235294 0.5137255; 0.6392157 0.61960787 … 0.56078434 0.56078434]

[0.69411767 0.5372549 … 0.3529412 0.31764707; 0.627451 0.6 … 0.3137255 0.28627452; … ; 0.6039216 0.59607846 … 0.44705883 0.4745098; 0.5803922 0.5803922 … 0.5254902 0.52156866]

[0.73333335 0.53333336 … 0.2784314 0.27450982; 0.66

In [9]:
train = ([(cat(imgs[i]..., dims = 4), labels[:,i]) for i in partition(1:49000, 1000)]) |> gpu
valset = 49001:50000
valX = cat(imgs[valset]..., dims = 4) |> gpu
valY = labels[:, valset] |> gpu

10×1000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  1  1
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅

In [10]:
m = Chain(
  Conv((5,5), 3=>16, relu),
  MaxPool((2,2)),
  Conv((5,5), 16=>8, relu),
  MaxPool((2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(200, 120),
  Dense(120, 84),
  Dense(84, 10),
  softmax) |> gpu

Chain(
  Conv((5, 5), 3 => 16, relu),          # 1_216 parameters
  MaxPool((2, 2)),
  Conv((5, 5), 16 => 8, relu),          # 3_208 parameters
  MaxPool((2, 2)),
  var"#7#8"(),
  Dense(200, 120),                      # 24_120 parameters
  Dense(120, 84),                       # 10_164 parameters
  Dense(84, 10),                        # 850 parameters
  NNlib.softmax,
)                   # Total: 10 arrays, 39_558 parameters, 155.852 KiB.

In [11]:
using Flux: crossentropy, Momentum

loss(x, y) = sum(crossentropy(m(x), y))
opt = Momentum(0.01)

Momentum(0.01, 0.9, IdDict{Any, Any}())

In [12]:
accuracy(x, y) = mean(onecold(m(x), 1:10) .== onecold(y, 1:10))


accuracy (generic function with 1 method)

In [13]:
epochs = 10

for epoch = 1:epochs
  for d in train
    gs = gradient(params(m)) do
      l = loss(d...)
    end
    update!(opt, params(m), gs)
  end
  @show accuracy(valX, valY)
end

accuracy(valX, valY) = 0.232
accuracy(valX, valY) = 0.272
accuracy(valX, valY) = 0.378
accuracy(valX, valY) = 0.401
accuracy(valX, valY) = 0.435
accuracy(valX, valY) = 0.457
accuracy(valX, valY) = 0.467
accuracy(valX, valY) = 0.473
accuracy(valX, valY) = 0.5
accuracy(valX, valY) = 0.512


In [14]:
valset = valimgs(CIFAR10)
valimg = [getarray(valset[i].img) for i in 1:10000]
labels = onehotbatch([valset[i].ground_truth.class for i in 1:10000],1:10)
test = gpu.([(cat(valimg[i]..., dims = 4), labels[:,i]) for i in partition(1:10000, 1000)])

10-element Vector{Tuple{Array{Float32, 4}, Flux.OneHotArray{UInt32, 10, 1, 2, Vector{UInt32}}}}:
 ([0.61960787 0.62352943 … 0.49411765 0.45490196; 0.59607846 0.5921569 … 0.49019608 0.46666667; … ; 0.23921569 0.19215687 … 0.11372549 0.078431375; 0.21176471 0.21960784 … 0.13333334 0.08235294]

[0.4392157 0.43529412 … 0.35686275 0.33333334; 0.4392157 0.43137255 … 0.35686275 0.34509805; … ; 0.45490196 0.4 … 0.32156864 0.2509804; 0.41960785 0.4117647 … 0.32941177 0.2627451]

[0.19215687 0.18431373 … 0.14117648 0.12941177; 0.2 0.15686275 … 0.1254902 0.13333334; … ; 0.65882355 0.5803922 … 0.49411765 0.41960785; 0.627451 0.58431375 … 0.5058824 0.43137255]

[0.92156863 0.90588236 … 0.9137255 0.9098039; 0.93333334 0.92156863 … 0.9254902 0.92156863; … ; 0.32156864 0.18039216 … 0.7254902 0.73333335; 0.33333334 0.24313726 … 0.7058824 0.7294118]

[0.92156863 0.90588236 … 0.9137255 0.9098039; 0.93333334 0.92156863 … 0.9254902 0.92156863; … ; 0.3764706 0.22352941 … 0.78431374 0.7921569; 0.39607844 0.2

In [ ]:
ids = rand(1:10000, 10)
image.(valset[ids])

In [16]:
rand_test = getarray.(image.(valset[ids]))
rand_test = cat(rand_test..., dims = 4) |> gpu
rand_truth = ground_truth.(valset[ids])
m(rand_test)

10×10 Matrix{Float32}:
 0.0269649    0.0895869   0.00156005   …  0.000801428  0.0033084
 0.000568245  0.154636    7.31146f-5      0.000305377  1.33458f-5
 0.252885     0.0101299   0.0788518       0.0382378    0.0916858
 0.090282     0.0180719   0.105265        0.352767     0.0232188
 0.267004     0.0110116   0.172252        0.0233806    0.10715
 0.044843     0.00893984  0.0791501    …  0.537332     0.00335127
 0.28087      0.033008    0.543526        0.0283682    0.766584
 0.0246912    0.0198171   0.018083        0.0173325    0.000493203
 0.00949219   0.152308    0.000238693     0.000204157  0.00387179
 0.00239962   0.502492    0.00100075      0.00127098   0.000323074

In [17]:
class_correct = zeros(10)
class_total = zeros(10)
for i in 1:10
  preds = m(test[i][1])
  lab = test[i][2]
  for j = 1:1000
    pred_class = findmax(preds[:, j])[2]
    actual_class = findmax(lab[:, j])[2]
    if pred_class == actual_class
      class_correct[pred_class] += 1
    end
    class_total[actual_class] += 1
  end
end

class_correct ./ class_total

10-element Vector{Float64}:
 0.547
 0.575
 0.397
 0.252
 0.36
 0.447
 0.682
 0.528
 0.663
 0.529